## Commutation

A BLDC motors contains two main components of interest, a rotor (the rotating part) and a stator (the stationary part). Depending on the type of blc motor, the magnets will be either (1) attached permanently to the rotor with the coils on the stator (outrunner) or (2) attached permanently to the  stator with the coils on the rotor (inrunner).

The goal of controlling a BLDC motor is to control current to create a rotating magnetic field on the coils that is always orthogonal to the permanent magnetic field such that we generate the highest-efficiency torque to cause the motor to turn. (Any angle other than $90^\circ$ is inefficient and will result in wasted energy disipated as heat.

The process of controlling the motor current which creates the magnetic field of the motor in such a way that causes the motor to spin is called *commutation*. With a *brushed* DC motor, this process is done mechanically where the physical brushes on the motor shaft physically switch coils based on the angle of the motor. With a *brushless* DC motor, this process must be done with sensors and a microcontroller.

To perform commutation, there are two things we need to measure at all times: (1) the angle of the motor shaft, which lets us compute the angle of the permanent magnetic field, and (2) the actual current going through each of the three phase windings. There is one (technically 3) thing we need to control at all times: the current going through each of the three phase windings.


## Background

A BLDC motor is a 3 phase system, and thinking about things in terms of each phase can get confusing. That's where the Parke and Clark Transforms come into play. They simplify the way to conceptualize what exactly we are trying to control. The result is that we can think about both current and the the motor's magnetic field as 2D vectors with angle and magnitude.


## The Parke and Clarke Transform

Let's collect the 3-phase currents into a vector $\begin{bmatrix}a \\ b \\ c \\ \end{bmatrix}$ The vector exists in a 3D "coil space" spanned by a linear combination of $a$, $b$, and $c$ orthogonal components. (See [Wikipedia](https://en.wikipedia.org/wiki/Direct-quadrature-zero_transformation#/media/File:DQZ_1.svg))


Let the Clarke Transform be denoted $K_c$. This transform rotates our coil space so that we're looking at it top-down from the tip of a $\begin{bmatrix} 1\\ 1\\ 1\\ \end{bmatrix}$ vector in that space. (Again, see [Wikipedia](https://en.wikipedia.org/wiki/Direct-quadrature-zero_transformation#/media/File:DQZ_6.svg)). After doing this rotation, we can basically ignore the Z component, making our system 2D.


Let the Parke Transform be denoted $K_p$. This transform takes vectors from our 2D space and rotates them about the z axis such that the new x axis of this space is always aligned with the motor's magnetic field flux vector. This space will rotate in time, but since we know the angle of the motor shaft at all times, we can do calculations from the perspective of this rotating space as if they weren't rotating.


In Linear Algebra terms, then the Parke and Clarke Transforms are just a change of basis.

$$
\begin{bmatrix} d \\ q \\ z \end{bmatrix} = 
K_{p} K_{c} \begin{bmatrix} a \\ b\\ c\\ \end{bmatrix} = 
K_{cp} \begin{bmatrix} a\\ b\\ c\\ \end{bmatrix}
$$

Note that $K_{cp}$ is invertible.


## Terms

Recall that torque is directly proportional to current. ([Wikipedia](https://en.wikipedia.org/wiki/Motor_constants))
$$
T = K_T i
$$

Where $K_T = \frac{1}{K_v}$, where $K_v$ might be a more familiar motor specification.

Therefore, building a current controller is equivalent to building a torque controller.



## A Simplified Algorithm

1. Determine (or set) magnetic field orientation. (D)
1. Use D to determine Q in transformed frame.
1. Set the Magnitude of Q according to desired torque
1. Inverse Transform Q into 3 coils currents
1. Apply currents to motor.
1. Wait some small time delta.
1. Measure actual currents. Measure D from the rotor angle.
1. Forward-Transform currents into D in the Transformed frame.
1. Use new D measurement to compute desired Q current vector.
1. Use actual currents to compute actual Q current vector.
1. Feed Error into PI controller.
1. Repeat from step 5


## Pole Pairs

In reality, BLDC motors have multiple pole pairs per revolution. And to smooth out motor cogging, the number of poles on the stator is not equal to the number of poles on the rotor. The crux is that the magnetic field angle that we've described above is not the same as the rotor angle. But the conversion is simple and dictated by the [winding combination](https://electronics.stackexchange.com/questions/483177/how-to-determine-brushless-dc-bldc-winding-pattern-based-on-poles-and-slots?rq=1)

Assuming a 14P12N BLDC motor (fairly common) there are 7 electrical cycles per mechanical cycle.


## Resources
1. [Field Oriented Control of Permanent Magnet Motors](https://www.youtube.com/watch?v=cdiZUszYLiA&t=347s&ab_channel=TexasInstruments)
1. [Janzen Lee](https://www.youtube.com/watch?v=mbJOxqxLkLE&ab_channel=JantzenLee)
1. [SUBMS: Motor Modeling](http://subms.com/tutorials/motor_modeling)
1. [SUMBS: Motor Control](http://subms.com/tutorials/motor_control)
1. [Park and Clark Transform Summary](https://www.cypress.com/file/222111/download)